<a href="https://colab.research.google.com/github/AJAX6255/Rubin-DES-Photometric-Calibration/blob/main/Rubin_DES_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### (A) Setup and Data Loading

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table, join
from astropy.coordinates import SkyCoord

print("Loading DES data...")
des = Table.read("data/DES_DR2_sample.fits")
print("DES data loaded successfully.")
display(des[:5])

print("\nLoading Rubin data...")
rubin = Table.read("data/Rubin_DP1_sample.fits")
print("Rubin data loaded successfully.")
display(rubin[:5])

print("\nLoading Gaia data...")
gaia = Table.read("data/Gaia_DR3_reference.fits")  # Gaia as astrometric bridge
print("Gaia data loaded successfully.")
display(gaia[:5])

### (B) Cross-Matching with Gaia

In [ ]:
def crossmatch_with_gaia(cat, gaia, radius=0.5*u.arcsec):
    # Convert to SkyCoord objects
    coord_cat = SkyCoord(cat['RA'], cat['DEC'], unit='deg')
    coord_gaia = SkyCoord(gaia['ra'], gaia['dec'], unit='deg')

    # Match catalog to Gaia
    idx, sep, _ = coord_cat.match_to_catalog_sky(coord_gaia)
    matched_cat = cat[sep < radius]

    return matched_cat

# Example: Match DES and Rubin to Gaia
des_matched = crossmatch_with_gaia(des, gaia)
rubin_matched = crossmatch_with_gaia(rubin, gaia)

### (C) Photometric Transformation Fitting

In [ ]:
from scipy.optimize import curve_fit
import warnings

def fit_photometric_transform(des_matched, rubin_matched, band):
    """
    Fits a photometric transformation between DES and Rubin magnitudes for a given band.

    Args:
        des_matched (Table): Astropy Table with DES data matched to Gaia.
        rubin_matched (Table): Astropy Table with Rubin data matched to Gaia.
        band (str): The photometric band to fit (e.e. 'r', 'g', 'i', etc.).

    Returns:
        tuple: A tuple containing the optimized parameters (a, b, c) and their covariance matrix.
               Returns (None, None) if fitting fails.
    """
    def photometric_transform(mag_des, color_des, a, b, c):
        return a * mag_des + b * color_des + c

    try:
        # Calculate color and magnitude difference for the given band
        color_des = des_matched['g_mag'] - des_matched['r_mag'] # Assuming g-r color
        mag_diff = rubin_matched[f'{band}_mag'] - des_matched[f'{band}_mag']
        mag_des = des_matched[f'{band}_mag']
        mag_des_err = des_matched[f'{band}_mag_err']

        # Perform the curve fit, ignoring potential warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            popt, pcov = curve_fit(photometric_transform,
                                   mag_des,
                                   mag_diff,
                                   sigma=mag_des_err)

        print(f"Fitting for {band}-band:")
        print(f"Zeropoint: {popt[2]:.3f}, Color term: {popt[1]:.3f}")
        return popt, pcov

    except Exception as e:
        print(f"Fitting failed for {band}-band: {e}")
        return None, None

# Example usage: Fit for r-band
popt_r, pcov_r = fit_photometric_transform(des_matched, rubin_matched, 'r')

# Example usage: Fit for g-band
# popt_g, pcov_g = fit_photometric_transform(des_matched, rubin_matched, 'g')

### (D) Visualization

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6)) # Optional: Set figure size
plt.scatter(des_matched['r_mag'] - des_matched['i_mag'],
            rubin_matched['r_mag'] - des_matched['r_mag'],
            s=1, alpha=0.5)
plt.xlabel("DES (r-i)")
plt.ylabel("Rubin r - DES r")
plt.title("Photometric Transformation: Rubin r vs DES (r-i)") # Add a title
plt.grid(True) # Ensure grid is visible

# Optional: Save the plot to a file
# plt.savefig("photometric_transformation_r_band.png")

plt.show() # Display the plot